<a href="https://colab.research.google.com/github/Prajna1999/fithub/blob/main/fithub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## **Fithub Notebook Documentation**

### **1. Overview**
`fithub.ipynb` is a comprehensive notebook hosted on GitHub, designed to provide tools and methods related to fitness queries. Leveraging powerful libraries such as `openai` and `llama_index`, the notebook offers capabilities to understand and process natural language queries about fitness.

---

### **2. Getting Started**

#### **2.1. Accessing the Notebook**
The notebook can be seamlessly accessed and executed in Google Colab using the provided link:
[Open in Google Colab](https://colab.research.google.com/github/Prajna1999/Tools/blob/master/fithub.ipynb)

---

### **3. Initial Setup**

#### **3.1. Package Installation**
Before diving into the functionalities, ensure the `llama-index` package is installed:
```python
!pip install llama-index
```

#### **3.2. API Configuration**
For seamless integration with OpenAI, ensure you import the necessary libraries and initialize your API key:
```python
import openai
from getpass import getpass
openai.api_key = getpass("Enter your openai key: ")
```

---

### **4. Libraries and Modules**

The notebook heavily relies on various libraries and modules. Here's a breakdown:

- **OpenAI & Llama Index**: Provides the backbone for processing natural language queries.
- **SQLAlchemy**: Used for setting up the database and ORM structures.
- **Nest Asyncio**: Helps in handling asynchronous tasks.
- **Logging**: Assists in debugging and logging information.

---

### **5. Data Structures and Initializations**

#### **5.1. Llama Index and OpenAI Configuration**
The notebook sets up various components of the `llama_index` library, such as the node parser and LLM, to integrate with OpenAI's capabilities.

#### **5.2. Database and ORM**
Using SQLAlchemy, the notebook defines tables and relationships for fitness exercises, tags, and associations between them.

---

### **6. Query Engines**

The notebook boasts powerful query engines that translate natural language queries:

- **SQL Tool**: Handles queries related to exercises, tags, and their relationships.
- **Semantic Engine Tool**: Answers semantic questions about exercises and fitness.
- **Join Query Engine**: Combines the capabilities of the above tools for a comprehensive query solution.

---

### **7. User Interaction**

The `execute_query` function provides an interactive way for users to input their queries, which are then processed using the configured query engines.

---

### **8. Debugging and Logging**

For developers, the notebook sets up logging configurations for better output handling and debugging, ensuring clarity during troubleshooting.

---

### **9. Conclusion**

`fithub.ipynb` is a powerful notebook for anyone interested in processing and understanding fitness-related queries. By integrating advanced libraries and providing user-friendly tools, it stands as a valuable resource in the domain of fitness data processing.





In [ ]:
!pip install llama-index


In [ ]:
import openai
from getpass import getpass
openai.api_key=getpass("Enter your openai key: ")


Enter your openai key: ··········


In [ ]:
from llama_index import(
    VectorStoreIndex,
    SimpleDirectoryReader,ServiceContext,
    StorageContext,SQLDatabase, WikipediaReader
)

In [ ]:
from llama_index.node_parser.simple import SimpleNodeParser
from llama_index import ServiceContext, LLMPredictor
from llama_index.storage import StorageContext
from llama_index.vector_stores import PineconeVectorStore
from llama_index.text_splitter import TokenTextSplitter
from llama_index.llms import OpenAI

In [ ]:
#define the node parse and LLM
chunk_size=1024
llm=OpenAI(temperature=0, model="gpt-3.5-turbo-16k", streaming=True)
service_context=ServiceContext.from_defaults(chunk_size=chunk_size, llm=llm)
# text_splitter=TokenTextSplitter(chunk_size=chunk_size)
# node_parser=SimpleNodeParser(text_splitter=text_splitter)

In [ ]:
!pip install transformers

In [ ]:
# #define pinecone vector storeindex
# vector_store=PineconeVectorStore(
#     pinecone_index=pinecone_index, namespace="fitness_guide"
# )

# storage_context=StorageContext.from_defaults(vector_store=vector_store)
# vector_index=VectorStoreIndex([], storage_context=storage_context)

In [ ]:
from sqlalchemy import create_engine, Table, Column, Integer, String, ForeignKey, MetaData
# from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Mapped, relationship, DeclarativeBase

metadata_obj=MetaData()
class Base(DeclarativeBase):
  metadata_obj


# Association table for M:N relationships
exercise_tag = Table(
    "exercise_table",
    Base.metadata,
    Column("left_id", Integer, ForeignKey("exercise.id")),
    Column("right_id", Integer, ForeignKey("tag.id")),
)

class Exercise(Base):
    __tablename__ = "exercise"

    id: Mapped[int] = Column(Integer, primary_key=True)
    exercise_name: Mapped[str] = Column(String(30))
    exercise_description: Mapped[str] = Column(String(255))
    tags: Mapped['Tag'] = relationship('Tag', secondary=exercise_tag)


    def __repr__(self) -> str:
        return f"Exercise(id={self.id!r}, name={self.exercise_name!r}, description={self.exercise_description!r})"

class Tag(Base):
    __tablename__ = "tag"
    id: Mapped[int] = Column(Integer, primary_key=True)
    exercise_id:Mapped[int]=Column(ForeignKey("exercise.id"))
    tag_name: Mapped[str] = Column(String(30))

    def __repr__(self) -> str:
        return f"Tag(id={self.id!r}, name={self.tag_name!r})"

engine = create_engine("sqlite:///:memory:", future=True) # Adjust this line to your database connection
print(Base.metadata.create_all(engine))


None


In [ ]:
# Create a session
from sqlalchemy.orm import sessionmaker

# Create a session factory
Session = sessionmaker(bind=engine)

# Create a session
session = Session()

# Define exercises and corresponding tags
exercises_with_tags = [
    ("crunches", ["abs", "core"]),
    ("pushups", ["upperbody", "chest"]),
    ("lifting", ["strength", "upperbody"]),
    ("squats", ["lowerbody", "legs"]),
    ("planks", ["core", "fullbody"]),
]

# Insert the tags first and keep a reference to them
tag_objects = {}
for exercise, tags in exercises_with_tags:
    for tag in tags:
        if tag not in tag_objects:
            tag_obj = Tag(tag_name=tag)
            session.add(tag_obj)
            tag_objects[tag] = tag_obj

# Insert the exercises and associate them with the corresponding tags
for exercise_name, tags in exercises_with_tags:
    exercise = Exercise(
        exercise_name=exercise_name,
        exercise_description=f"Description for {exercise_name}",
        # tags=[tag_objects[tag] for tag in tags]
    )
    session.add(exercise)

# Commit the changes to the database
session.commit()

# Optional: Query the tables to verify the inserted data
exercises = session.query(Exercise).all()
tags = session.query(Tag).all()

for exercise in exercises:
    print(exercise)
    # print("Tags:", [tag.tag_name for tag in exercise.tags])

for tag in tags:
    print(tag)

# Close the session
session.close()


Exercise(id=1, name='crunches', description='Description for crunches')
Exercise(id=2, name='pushups', description='Description for pushups')
Exercise(id=3, name='lifting', description='Description for lifting')
Exercise(id=4, name='squats', description='Description for squats')
Exercise(id=5, name='planks', description='Description for planks')
Tag(id=1, name='abs')
Tag(id=2, name='core')
Tag(id=3, name='upperbody')
Tag(id=4, name='chest')
Tag(id=5, name='strength')
Tag(id=6, name='lowerbody')
Tag(id=7, name='legs')
Tag(id=8, name='fullbody')


In [ ]:
from pathlib import Path
from llama_index import download_loader

PDFReader = download_loader("PDFReader")

loader = PDFReader()
documents = loader.load_data(file=Path(f'/content/fitness-guide.pdf'))

/usr/lib/python3.10/typing.py:1512: RuntimeWarning: coroutine 'SubQuestionQueryEngine._aquery_subq' was never awaited
  return super().__instancecheck__(instance)
/usr/lib/python3.10/typing.py:1512: RuntimeWarning: coroutine 'run_async_tasks.<locals>._gather' was never awaited
  return super().__instancecheck__(instance)


In [ ]:
print(documents)

[Document(id_='4d78a481-cb16-4152-b001-bb2d2b59660b', embedding=None, metadata={'page_label': '1', 'file_name': 'fitness-guide.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='074371c91fb0edb326b3e979089bc5796d93d16d41f875662b50e546f081f91e', text='', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='8d25eb7f-37bd-4597-90da-84f89012a64f', embedding=None, metadata={'page_label': '2', 'file_name': 'fitness-guide.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='afe1e454e4ac6a33bf93490920e03db6e809146ee603fefec605ecb132a099ad', text=' \n  ', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), Document(id_='e67a8b02-c15d-4edd-b61e-4d56a88c71c1', embedding=None, metadata={'page_label': '3', 'file_na

In [ ]:
#build an SQL index
sql_database=SQLDatabase(engine, include_tables=["exercise", "tag"])
print(sql_database)

In [ ]:
exercises=["crunches", "pushpups", "lifting", "squats", "planks", "lifting"]

In [ ]:
#Build Vector Index
vector_indices={}
vector_query_engines={}


for exercise in zip(exercises):
  vector_index=VectorStoreIndex.from_documents(documents)
  query_engine=vector_index.as_query_engine(similarity_top_k=5)
  vector_indices[exercise]=vector_index
  vector_query_engines[exercise]=query_engine




In [ ]:
print(vector_query_engines)

{('crunches',): <llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine object at 0x7c7126f760e0>, ('pushpups',): <llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine object at 0x7c7132d345e0>, ('lifting',): <llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine object at 0x7c7132998ac0>, ('squats',): <llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine object at 0x7c71329981c0>, ('planks',): <llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine object at 0x7c7126f777f0>}


In [ ]:
#define queries set as tools
from llama_index.query_engine import SQLJoinQueryEngine, RetrieverQueryEngine
from llama_index.tools.query_engine import QueryEngineTool
from llama_index.tools import ToolMetadata
from llama_index.indices.vector_store import VectorIndexAutoRetriever
from llama_index.query_engine import SubQuestionQueryEngine
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine

In [ ]:
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine

sql_query_engine=NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["exercise", "tag"],
)

In [ ]:
from llama_index.query_engine import SubQuestionQueryEngine
query_engine_tools=[]
for exercise in exercises:
  query_engine=vector_query_engines[exercise,]

  query_engine_tool=QueryEngineTool(
      query_engine=query_engine,
      metadata=ToolMetadata(
          name=exercise, description=f"Provides information about the {exercise}"
      ),
  )
  query_engine_tools.append(query_engine_tool)

In [ ]:
s_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)

In [ ]:
sql_tool=QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over  tables containing: "
        "exercises,tags, exercise_tags containing various exercises and their corresponding tags"
    ),
)
s_engine_tool=QueryEngineTool.from_defaults(
    query_engine=s_engine,
    description=f"Useful for answering semantic questions about different exercises, bodyparts and general fitness related queries",
)

In [ ]:
#define SQLJOinQueryEngine
query_engine=SQLJoinQueryEngine(
    sql_tool, s_engine_tool, service_context=service_context
)

In [ ]:
import nest_asyncio

nest_asyncio.apply()

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
def execute_query():
    try:
        query = input("Please enter your query: ")

        if not query or not isinstance(query, str):
            return "Error: The query must be a non-empty string."

        if not hasattr(query_engine, 'query') or not callable(query_engine.query):
            return "Error: query_engine does not have a callable 'query' method."

        response = query_engine.query(query.strip())

        if not response:
            return "Error: The query returned no results."

        return response
    except Exception as e:
        return f"An error occurred while executing the query: {str(e)}"

In [169]:
try:
  response=execute_query()
  print(response)
except Exception as e:
  print(f"An error occurred while executing the query: {str(e)}")

Please enter your query: I only have a cattle bell and a dumb bell.  I would like to blend my exercise with Yoga. Can you give me a workout?
Querying other query engine: The question is about blending exercise with Yoga and asking for a workout.
Generated 5 sub questions.
[crunches] Q: What are some yoga exercises that can be done with a cattle bell?
[pushpups] Q: What are some yoga exercises that can be done with a dumb bell?
[lifting] Q: What are some yoga exercises that can be done with a cattle bell?
[squats] Q: What are some yoga exercises that can be done with a dumb bell?
[planks] Q: What are some yoga exercises that can be done with a cattle bell?
[planks] A: I'm sorry, but there is no information in the given context about yoga exercises that can be done with a cattle bell.
[crunches] A: I'm sorry, but there is no information in the given context about yoga exercises that can be done with a cattle bell.
[lifting] A: I'm sorry, but there is no information in the given context a

In [170]:
print(response)

Yes, you can incorporate both the cattle bell and the dumbbell into your yoga-inspired workout. Some exercises you can try include the Warrior pose with the dumbbell held in one hand, the Forward Bend with the cattle bell held in both hands, the Lightning Bolt with the dumbbell held overhead, and the Mountain pose with the cattle bell held at your chest. These exercises will help you blend strength training with yoga movements for a well-rounded workout.


In [ ]:
execute_query()

Please enter your query: Recent Popularity of Tao Be
Querying other query engine: The question is about the recent popularity of Tao Be, which is related to general fitness and exercise queries.
Generated 5 sub questions.
[crunches] Q: What is the recent popularity of Tao Be according to crunches?
[pushpups] Q: What is the recent popularity of Tao Be according to pushpups?
[lifting] Q: What is the recent popularity of Tao Be according to lifting?
[squats] Q: What is the recent popularity of Tao Be according to squats?
[planks] Q: What is the recent popularity of Tao Be according to planks?
[pushpups] A: According to the given context information, there is no mention of "Tao Be" or any similar term. Therefore, there is no information available about the recent popularity of "Tao Be" based on the provided context.
[planks] A: Billy Blanks, the creator of Tae Bo, has stated that Tae Bo has gained recent popularity and is now in an unprecedented high position in the fitness world. It has b

Response(response='There is no information available about the recent popularity of "Tao Be" based on the provided context.', source_nodes=[NodeWithScore(node=TextNode(id_='ae0c2e31-aab6-49f8-958b-acd292575c8c', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='7c3bd9d486f36836e37f38427b331994d1d9eb629b34cf2cc2caf323211d793c', text='Sub question: What is the recent popularity of Tao Be according to crunches?\nResponse: The recent popularity of Tae Bo, according to the information provided, is that it has become a fitness craze and a hot video workout. It was developed by martial artist Bill Blanks and combines karate kicks and boxing jabs. It has gained popularity through the sale of millions of videotapes and has been embraced by Hollywood stars and sports stars. Tae Bo is known for its intense, full-body moves and is said to be as intense as step aerobics.', start_char_idx=None, end_char_idx=None, text_template='{meta